In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
from sklearn import datasets
digits = datasets.load_digits()

In [3]:
print(len(digits.data))
print(digits.target.size)

1797
1797


In [4]:
from sklearn.datasets import fetch_openml
wine = fetch_openml(name='wine', version=1)

In [5]:
print(len(wine.data))
print(wine.target.size)

178
178


In [6]:
# 5-fold cross validation for digits dataset

digitsTrainingSetSize = int(np.ceil(0.8 * len(digits.data)))
digitsValidationSetSize = int(len(digits.data) - digitsTrainingSetSize)

xDigitsTrainingSets = []
yDigitsTrainingSets = []
xDigitsValidationSets = []
yDigitsValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []

    for index, data in enumerate(digits.data[foldIndex*digitsValidationSetSize:((foldIndex*digitsValidationSetSize)+digitsValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(digits.target[index])
    
    xTrainingSet = []
    yTrainingSet = []

    for index, data in enumerate(digits.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(digits.target[index])
            
    xDigitsTrainingSets.append(xTrainingSet)
    yDigitsTrainingSets.append(yTrainingSet)
    xDigitsValidationSets.append(xValidationSet)
    yDigitsValidationSets.append(yValidationSet)
    
# 5-fold cross validation for wine dataset

wineTrainingSetSize = int(np.ceil(0.8 * len(wine.data)))
wineValidationSetSize = int(len(wine.data) - wineTrainingSetSize)

xWineTrainingSets = []
yWineTrainingSets = []
xWineValidationSets = []
yWineValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []

    for index, data in enumerate(wine.data[foldIndex*wineValidationSetSize:((foldIndex*wineValidationSetSize)+wineValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(wine.target[index])
    
    xTrainingSet = []
    yTrainingSet = []

    for index, data in enumerate(wine.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(wine.target[index])
            
    xWineTrainingSets.append(xTrainingSet)
    yWineTrainingSets.append(yTrainingSet)
    xWineValidationSets.append(xValidationSet)
    yWineValidationSets.append(yValidationSet)
    

In [7]:
# one-hot encoding of y for digits dataset

numberOfDigitsTargets = 10
numberOfWineTargets = 3

for index, fold in enumerate(yDigitsTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yDigitsValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsValidationSets[index] = encodedFold

# one-hot encoding of y for wine dataset

for index, fold in enumerate(yWineTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yWineValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineValidationSets[index] = encodedFold


In [8]:
def getRandomIndices(arr, batch_size):
    indices = []
    
    if batch_size > len(arr):
        print("Error: batch size larger than size of dataset.")
        return
    
    while batch_size > 0:
        x = np.floor(np.random.random() * len(arr))
        if x not in indices:
            indices.append(int(x))
            batch_size -= 1
    
    return indices

In [9]:
# gradient descent class
 
class GradientDescent:
    
    def __init__(self, batch_size, learning_rate=.001, momentum=0.9, max_iters=10, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.batch_size = batch_size
        self.max_iters = max_iters
        self.epsilon = epsilon
            
    def run(self, gradient_fn, x, y, w):
        # TODO: grad variable and usage in loop?
        
        grad = np.inf
        t = 1
        
        while np.linalg.norm(grad) > self.epsilon and t < self.max_iters:
            gradients = gradient_fn(x, y, w, self.batch_size)
            
            for c in range(len(y[0])):
                w[c] = w[c] - self.learning_rate * gradients[c]
                print("w: ", w[c])
                
            t += 1
        
        return w

In [10]:
# logistic regression

class LogisticRegression:
    def __init__(self, add_bias=True):
        self.add_bias = add_bias
        pass
            
    def fit(self, x, y, optimizer):
        # TODO: add bias
        
        def gradient(x, y, w, batch_size):
            gradients = np.zeros(len(w)).tolist()

            indices = getRandomIndices(x, batch_size)
            print(indices)

            for index in indices:
                a = np.asarray(x[index])
                b = np.asarray(y[index])

                for c in range(len(b)):
                    w_x =  w[c] @ a
                    num = np.exp(w_x)

                    den = 0
                    for i in range(len(b)):
                        w_x =  w[i] @ a
                        den += np.exp(w_x)

                    yh_c = num/den

                    y_c = b[c]
                    cost_c = np.dot(yh_c - y_c, a)
                    
                    if cost_c[0] == 0:
                        print("x", a)
                        print("y", b)
                        print(c)
                        print(num)
                        print(den)
                        print(yh_c)
                        print(y_c)
                        print("gradient", cost_c)
                    

                    gradients[c] += cost_c

            return gradients
        
        w0 = []
        for c in range(len(y[0])):
            w0.append(np.zeros(len(x[0])))
            
        self.w = optimizer.run(gradient, x, y, w0)
    
    def predict(self, x):
        if self.add_bias:
            x = np.column_stack([x,np.ones(N)])
        yh = x@self.w
        return yh

In [11]:
gradientDescentModel = GradientDescent(5)
logisticRegressionModel = LogisticRegression(False)


In [12]:
logisticRegressionModel.fit(xWineTrainingSets[0], yWineTrainingSets[0], gradientDescentModel)





#xWineTrainingSets = []
#yWineTrainingSets = []
#xWineValidationSets = []
#yWineValidationSets = []

[60, 88, 50, 92, 53]
w:  [-2.05400000e-02 -4.16333333e-03 -3.93666667e-03 -3.62000000e-02
 -1.74333333e-01 -3.80000000e-03 -3.59666667e-03 -6.60000000e-04
 -3.28666667e-03 -4.54000000e-03 -1.69666667e-03 -4.69333333e-03
 -9.71000000e-01]
w:  [4.10800000e-02 8.32666667e-03 7.87333333e-03 7.24000000e-02
 3.48666667e-01 7.60000000e-03 7.19333333e-03 1.32000000e-03
 6.57333333e-03 9.08000000e-03 3.39333333e-03 9.38666667e-03
 1.94200000e+00]
w:  [-2.05400000e-02 -4.16333333e-03 -3.93666667e-03 -3.62000000e-02
 -1.74333333e-01 -3.80000000e-03 -3.59666667e-03 -6.60000000e-04
 -3.28666667e-03 -4.54000000e-03 -1.69666667e-03 -4.69333333e-03
 -9.71000000e-01]
[22, 4, 56, 84, 87]
x [1.329e+01 1.970e+00 2.680e+00 1.680e+01 1.020e+02 3.000e+00 3.230e+00
 3.100e-01 1.660e+00 6.000e+00 1.070e+00 2.840e+00 1.270e+03]
y [1. 0. 0.]
2
0.0
inf
0.0
0.0
gradient [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
x [1.422e+01 3.990e+00 2.510e+00 1.320e+01 1.280e+02 3.000e+00 3.040e+00
 2.000e-01 2.080e+00 5.100e+00 8

<ipython-input-10-1ba5324c8e47>:28: RuntimeWarning: overflow encountered in exp
  den += np.exp(w_x)
<ipython-input-10-1ba5324c8e47>:23: RuntimeWarning: overflow encountered in exp
  num = np.exp(w_x)
<ipython-input-10-1ba5324c8e47>:30: RuntimeWarning: invalid value encountered in double_scalars
  yh_c = num/den
